In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def corr2d(X, K):
    '''计算二维互相关运算'''
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### 卷积层

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__initx__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

#### 图像中目标的边缘检测

构造一个6 × 8像素的黑白图像。中间四列为黑色(0)，其余像素为白色(1)。

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

构造一个高度为1、宽度为2的卷积核K。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [6]:
K = torch.tensor([[1.0, -1.0]])

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

输出Y中的1代表从白色到黑色的边缘，‐1代表从黑色到白色的边缘，其他情况的输出为0。

这个卷积核K只可以检测垂直边缘，无法检测水平边缘。

In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### 学习卷积核

In [9]:
# 构造一个二维卷积层，它具有1个输出通道和形状为(1，2)的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式(批量大小、通道、高度、宽度)， 
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2 # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y - Y_hat) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if i % 2:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 10.619
epoch 4, loss 2.900
epoch 6, loss 0.944
epoch 8, loss 0.346
epoch 10, loss 0.135


In [10]:
# 卷积核权重非常接近我们之前定义的卷积核K
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9500, -1.0245]])

### 填充

卷积核大小通常为奇数，`padding`为$\frac{k - 1}{2}$

In [13]:
# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(con2d, X):
  # 这里的(1，1)表示批量大小和通道数都是1
  X = X.reshape((1, 1) + X.shape)
  Y = conv2d(X)
  return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [14]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 步幅

垂直步幅为$s_h$、水平步幅为$s_w$时，输出形状为

$$
\frac{n_h - k_h + p_h + s_h}{s_h} \times \frac{n_w - k_w + p_w + s_w}{s_w}
$$

1. 如果设置了$p_h = k_h - 1$和$p_w = k_w - 1$，则简化为

$$
\frac{n_h + s_h - 1}{s_h} \times \frac{n_w + s_w - 1}{s_w}
$$

2. 如果输入的高度和宽度可以被垂直和水平步幅整除

$$
\frac{n_h}{s_h} \times \frac{n_w}{s_w}
$$ 

通常$p_h = p_w$和$s_h = s_w$

In [16]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [17]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

### 多输入通道

In [25]:
def corr2d_multi_in(X, K):
  # 先遍历“X”和“K”的第0个维度(通道维度)，再把它们加在一起
  return sum(corr2d(x, k) for x, k in zip(X, K))

In [28]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], 
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])

K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 多输出通道

In [29]:
def corr2d_multi_in_out(X, K):
  # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
  # 最后将所有结果都叠加在一起
  return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [30]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [31]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])